# Week 1 Jupyter Notebook – Linear Regression 1
Each week, you will apply the concepts of that week to your Integrated Capstone Project’s dataset. In preparation for Milestone One, create a Jupyter Notebook (similar to in Module B, semester two) that illustrates these lessons. There are no specific questions to answer in your Jupyter Notebook files in this course; your general goal is to analyze your data, using the methods you have learned about in this course and in this program, and draw interesting conclusions. 

For Week 1, include concepts such as linear regression with polynomial terms, interaction terms, multicollinearity, variance inflation factor and regression, and categorical and continuous features. Complete your Jupyter Notebook homework by 11:59 pm ET on Sunday. 

In [ ]:
# Load Virtual Environment
!& "c:\Users\tbran\Python\repos\Semester 3 Repos\capstone\.venv\Scripts\Activate.ps1"


& was unexpected at this time.


In [28]:
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [11]:
project_path = 'C:/Users/tbran/Python/repos/Semester 3 Repos/capstone/'
data_path = project_path + 'data/'
src_path = project_path + 'src/'
model_path = project_path + 'models/'

In [12]:
file_name = 'MendeleyDelayData.csv'
df = pd.read_csv(data_path + file_name)

In [34]:
def optimize_dataframe(df, datetime_cols=None, fillna=False):
    """
    Cleans and optimizes a DataFrame:
    - Converts object datetime columns to datetime64
    - Converts object columns with repeated values to category
    - Downcasts numeric columns to smallest safe type
    - Optionally fills NaNs before downcasting
    
    Parameters:
        df (pd.DataFrame): The DataFrame to optimize
        datetime_cols (list): List of column names to convert to datetime
        fillna (bool): If True, fills NaNs before downcasting
    """
    
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage before optimization: {start_mem:.2f} MB")
    
    df = df.copy()
    
    # 1. Convert datetime columns
    if datetime_cols:
        for col in datetime_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # 2. Convert object columns to category if appropriate
    obj_cols = df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        num_unique = df[col].nunique()
        num_total = len(df[col])
        if num_unique / num_total < 0.5:  # heuristic: less than 50% unique
            df[col] = df[col].astype('category')
    
    # 3. Downcast numeric columns
    int_cols = df.select_dtypes(include=['int64', 'int32']).columns
    float_cols = df.select_dtypes(include=['float64', 'float32']).columns
    
    for col in int_cols:
        if fillna and df[col].isnull().any():
            df[col] = df[col].fillna(0)
        df[col] = pd.to_numeric(df[col], downcast='integer')
    
    for col in float_cols:
        if fillna and df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mean())
        df[col] = pd.to_numeric(df[col], downcast='float')
    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage after optimization: {end_mem:.2f} MB")
    print(f"Reduced by {100 * (start_mem - end_mem) / start_mem:.1f}%")
    
    return df


def clean_column_names(df, remove_accents=True):
    """
    Cleans DataFrame column names:
    - Strips whitespace
    - Converts to lowercase
    - Replaces spaces & special chars with underscores
    - Removes duplicate underscores
    - Optionally removes accents
    
    Parameters:
        df (pd.DataFrame): DataFrame whose columns to clean
        remove_accents (bool): If True, strips accents from characters
    
    Returns:
        pd.DataFrame: DataFrame with cleaned column names
    """
    def _clean(col):
        col = col.strip().lower()
        if remove_accents:
            col = ''.join(
                c for c in unicodedata.normalize('NFKD', col)
                if not unicodedata.combining(c)
            )
        col = re.sub(r'[^0-9a-zA-Z]+', '_', col)  # replace non-alphanumeric with _
        col = re.sub(r'_+', '_', col)             # collapse multiple underscores
        col = col.strip('_')                      # remove leading/trailing underscores
        return col
    
    df = df.copy()
    df.columns = [_clean(c) for c in df.columns]
    return df



def build_preprocessing_pipeline(df, target, 
                                  high_card_threshold=20, 
                                  scale_numeric=False):
    """
    Builds a preprocessing pipeline for linear regression:
    - One-hot encodes low-cardinality categorical columns
    - Target encodes high-cardinality categorical columns
    - Optionally scales numeric columns
    
    Parameters:
        df (pd.DataFrame): Input DataFrame (including target column)
        target (str): Name of target column
        high_card_threshold (int): Unique value cutoff for high-cardinality
        scale_numeric (bool): Whether to scale numeric features
        
    Returns:
        pipeline (ColumnTransformer): Preprocessing transformer
        low_card_cols (list): Low-cardinality categorical columns
        high_card_cols (list): High-cardinality categorical columns
        num_cols (list): Numeric columns
    """
    
    # Separate features and target
    X = df.drop(columns=[target])
    
    # Identify column types
    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    
    # Split categorical into low/high cardinality
    low_card_cols = [col for col in cat_cols if X[col].nunique() <= high_card_threshold]
    high_card_cols = [col for col in cat_cols if X[col].nunique() > high_card_threshold]
    
    # Transformers
    low_card_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    high_card_transformer = TargetEncoder()
    num_transformer = StandardScaler() if scale_numeric else 'passthrough'
    
    # Column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('low_card', low_card_transformer, low_card_cols),
            ('high_card', high_card_transformer, high_card_cols),
            ('num', num_transformer, num_cols)
        ]
    )
    
    return preprocessor, low_card_cols, high_card_cols, num_cols

import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df, features=None, vif_thresh=10.0):
    """
    Calculate Variance Inflation Factor (VIF) safely:
    - Removes constant columns
    - Removes perfectly collinear columns
    - Returns sorted VIF table
    
    Parameters:
        df (pd.DataFrame): DataFrame with numeric features
        features (list): Optional list of features to check; defaults to all numeric
        vif_thresh (float): Threshold for flagging high VIF
    
    Returns:
        pd.DataFrame: VIF table
    """
    # Select numeric columns if features not provided
    if features is None:
        features = df.select_dtypes(include=[np.number]).columns.tolist()
    
    X = df[features].copy()
    
    # 1. Drop constant columns
    constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
    if constant_cols:
        print(f"Dropping constant columns: {constant_cols}")
        X.drop(columns=constant_cols, inplace=True)
    
    # 2. Drop perfectly collinear columns
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    perfect_corr_cols = [col for col in upper.columns if any(upper[col] == 1.0)]
    if perfect_corr_cols:
        print(f"Dropping perfectly collinear columns: {perfect_corr_cols}")
        X.drop(columns=perfect_corr_cols, inplace=True)
    
    # 3. Calculate VIF
    X_const = X.assign(const=1)
    vif_data = pd.DataFrame({
        "feature": X.columns,
        "VIF": [variance_inflation_factor(X_const.values, i) for i in range(len(X.columns))]
    })
    
    # 4. Sort by VIF
    vif_data.sort_values(by="VIF", ascending=False, inplace=True)
    
    # 5. Flag high VIF
    vif_data["High_VIF"] = vif_data["VIF"] > vif_thresh
    
    return vif_data

def fit_polynomial_regression(X, y, degree=2):
    """
    Fits a polynomial regression model and returns the fitted model and transformed features.
    """
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    model.fit(X_poly, y)
    return model, poly

def add_interaction_terms(df, features):
    """
    Adds pairwise interaction terms between given features.
    """
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    interaction_array = poly.fit_transform(df[features])
    interaction_df = pd.DataFrame(interaction_array, columns=poly.get_feature_names_out(features))
    return pd.concat([df.reset_index(drop=True), interaction_df], axis=1)

def preprocess_features(df, categorical_cols, numeric_cols):
    """
    Returns a ColumnTransformer that one-hot encodes categorical columns
    and passes numeric columns through unchanged.
    """
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
            ('num', 'passthrough', numeric_cols)
        ]
    )
    return preprocessor

def regression_summary(X, y):
    """
    Fits an OLS regression model using statsmodels and prints the summary.
    """
    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()
    return model.summary()

In [14]:
df = optimize_dataframe(
    df,
    datetime_cols=['scheduleddepartdatetime'],
    fillna=True
)
df = clean_column_names(df)


Memory usage before optimization: 1008.24 MB
Memory usage after optimization: 150.66 MB
Reduced by 85.1%


In [20]:
# Get column categories

id_cols = ['originairportid', 'destairportid', ]
cat_cols = ['origin', 'dest', 'uniquecarrier', 'tailnum', 'origincityname', 'originstate', ]
date_cols = ['scheduleddepartdatetime', ]
target_cols = ['depdelay','arrdelay',]
feature_cols = [col for col in df.columns if col not in id_cols + cat_cols + date_cols + target_cols]

In [ ]:
# drop leakage columns for linear regression
df_lin = df.drop(columns=['arrdelay'] + id_cols + date_cols).copy()

preprocessor, low_card, high_card, num_cols = build_preprocessing_pipeline(
    df_lin, 
    target='depdelay', 
    high_card_threshold=20, 
    scale_numeric=True
)

print("Low-cardinality categorical:", low_card)
print("High-cardinality categorical:", high_card)
print("Numeric columns:", num_cols)

X = df_lin.drop(columns=['depdelay'])
y = df_lin['depdelay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model.fit(X_train, y_train)
print("R^2 score:", model.score(X_test, y_test))

Low-cardinality categorical: ['uniquecarrier']
High-cardinality categorical: ['origin', 'dest', 'tailnum', 'origincityname', 'originstate']
Numeric columns: ['marketshareorigin', 'marketsharedest', 'hhiorigin', 'hhidest', 'nonhubairportorigin', 'smallhubairportorigin', 'mediumhubairportorigin', 'largehubairportorigin', 'nonhubairportdest', 'smallhubairportdest', 'mediumhubairportdest', 'largehubairportdest', 'nonhubairlineorigin', 'smallhubairlineorigin', 'mediumhubairlineorigin', 'largehubairlineorigin', 'nonhubairlinedest', 'smallhubairlinedest', 'mediumhubairlinedest', 'largehubairlinedest', 'year', 'month', 'dayofmonth', 'dayofweek', 'scheduledhour', 'capacity', 'loadfactor', 'numflights', 'distance', 'monopolyroute', 'temperature', 'temp_ninfty_n10', 'temp_n10_0', 'temp_0_10', 'temp_10_20', 'temp_20_30', 'temp_30_40', 'temp_40_infty', 'windspeed', 'windspeedsquare', 'windgustdummy', 'windgustspeed', 'raindummy', 'raintracedummy', 'snowdummy', 'snowtracedummy', 'originmetropop', 'o

In [35]:
vif_table = calculate_vif(df)
print(vif_table)

c:\Users\tbran\Python\repos\Semester 3 Repos\capstone\.venv\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                    feature           VIF  High_VIF
13      largehubairportdest           inf      True
12     mediumhubairportdest           inf      True
10        nonhubairportdest           inf      True
11      smallhubairportdest           inf      True
7     smallhubairportorigin           inf      True
6       nonhubairportorigin           inf      True
18        nonhubairlinedest           inf      True
19      smallhubairlinedest           inf      True
16   mediumhubairlineorigin           inf      True
17    largehubairlineorigin           inf      True
20     mediumhubairlinedest           inf      True
21      largehubairlinedest           inf      True
15    smallhubairlineorigin           inf      True
14      nonhubairlineorigin           inf      True
8    mediumhubairportorigin  9.007199e+15      True
9     largehubairportorigin  9.007199e+15      True
39               temp_20_30  3.112541e+02      True
38               temp_10_20  2.888918e+02      True
37          